In [ ]:
import json
import sys
import hvplot.pandas
import pandas as pd
from utils import load_config, fetch_api_data, write_to_csv

In [ ]:
#https://www.ncdc.noaa.gov/swdiws
# 'nx3tvs'       - (Point)   NEXRAD Level-3 Tornado Vortex Signatures
# 'nx3meso'      - (Point)   NEXRAD Level-3 Mesocyclone Signatures
# 'nx3hail'      - (Point)   NEXRAD Level-3 Hail Signatures
# 'nx3structure' - (Point)   NEXRAD Level-3 Storm Cell Structure Information
# 'warn'         - (Polygon) Severe Thunderstorm, Tornado, Flash Flood and Special Marine warnings
datasets = ["nx3tvs"]
outputFormat = "json"
daterange = "20240701:20240731"  # "periodOfRecord"

for dataset in datasets:
    base_url = f"https://www.ncdc.noaa.gov/swdiws/{outputFormat}/{dataset}/{daterange}"
    filename = f"swdiws_{dataset}.csv"

    data = fetch_api_data(base_url)
    #print(json.dumps(data, indent=2))
    #print(data)

    if data and "result" in data:
        write_to_csv(data["result"], filename, "w")
    else:
        print(f"No data found or invalid response format for dataset: {dataset}")


In [ ]:
# 'nx3tvs'       - (Point)   NEXRAD Level-3 Tornado Vortex Signatures
# 'nx3meso'      - (Point)   NEXRAD Level-3 Mesocyclone Signatures
# 'nx3hail'      - (Point)   NEXRAD Level-3 Hail Signatures
# 'nx3structure' - (Point)   NEXRAD Level-3 Storm Cell Structure Information
# 'warn'         - (Polygon) Severe Thunderstorm, Tornado, Flash Flood and Special Marine warnings
datasets = ["nx3tvs"]
outputFormat = "geojson"
daterange = "20240701:20240731"  # "periodOfRecord"
numResults = 2500

# Initialize an empty list to store merged data
merged_data_list = []

for dataset in datasets:
    base_url = f"https://www.ncdc.noaa.gov/swdiws/{outputFormat}/{dataset}/{daterange}/{numResults}"
    filename = f"swdiws_{dataset}_{outputFormat}.csv"

    data = fetch_api_data(base_url)
    # Data is nested, retrieve "features" dictionary
    rows = data["features"]
    
    # Iterate over each record in rows as that is nested as well
    for record in rows:
       # Merge the 'properties' and 'geometry' dictionaries
        merged_data = {**record["properties"], **record["geometry"]}
        # Append the merged data to the list
        merged_data_list.append(merged_data)

    # Convert to DataFrame
    merged_df = pd.DataFrame(merged_data_list)

# Split the coordinates column into latitude and longitude
merged_df[['longitude', 'latitude']] = pd.DataFrame(merged_df['coordinates'].tolist(), index=merged_df.index)
# Drop the original coordinates column
merged_df = merged_df.drop(columns=['coordinates'])
merged_df.head(25)

    # if data and "features" in data:
    #     write_to_csv(data["features"], filename, "w")
    # else:
    #     print(f"No data found or invalid response format for dataset: {dataset}")

In [ ]:
# Convert 'MXDV' to numeric, forcing errors to NaN if conversion fails
merged_df['MXDV'] = pd.to_numeric(merged_df['MXDV'], errors='coerce')
# Ensure 'WSR_ID' is treated as a string
merged_df['WSR_ID'] = merged_df['WSR_ID'].astype(str)
merged_df.to_csv('swdiws_nx3tvs_geojson.csv', index=False)
print(merged_df.dtypes)

In [ ]:
# https://www.fema.gov/about/openfema/data-sets
base_url = f"https://www.fema.gov/api/open/v2/"

params = {"$count": "true",
          "$filter": "incidentType eq 'Tornado'"}

endpoint = "DisasterDeclarationsSummaries"
filename = f"{endpoint}.csv"
endpoint_url = f"{base_url}{endpoint}"

data = fetch_api_data(endpoint_url, params)
#print(json.dumps(data, indent=2))
tornado_summary_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])
write_to_csv(data["DisasterDeclarationsSummaries"], filename, "w")

disaster_numbers = tornado_summary_df.disasterNumber.unique()
formatted_disaster_numbers = ', '.join(f'{num}' for num in disaster_numbers)
formatted_disaster_numbers

    #disasterNumber
#tornado_newer_df = tornado_df[tornado_df["declarationDate"] > "2019-12-31"]
#tornado_df.groupby(by="disasterNumber").count()


In [ ]:
params = {"$count": "true",
          "$filter": f"disasterNumber in ({formatted_disaster_numbers})"}

endpoint = "HousingAssistanceOwners"
filename = f"{endpoint}.csv"
endpoint_url = f"{base_url}{endpoint}"
data = fetch_api_data(endpoint_url, params)
#print(json.dumps(data, indent=2))
housing_assistance_df = pd.DataFrame(data["HousingAssistanceOwners"])
write_to_csv(data["HousingAssistanceOwners"], filename, "w")
housing_assistance_df

In [ ]:
%%capture --no-display
# Configure the map plot
tornadoes_plot = merged_df.hvplot.points(
    "longitude",
    "latitude",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="MXDV",
    color="WSR_ID"
)

# Display the map
tornadoes_plot


In [ ]:
#DataCleaning

In [ ]:
merged_df

# Standardize column names to lowercase
merged_df.columns = [col.lower() for col in merged_df.columns]

#dropped cell and cell id columns
merged_df.drop(columns=['cell_type', 'cell_id',], inplace=True)

#renamed columns 
#max_shear = change in wind speed and direction with height in the atmosphere
#wsr_id = weather stations
#mxdv = maximum difference in velocity, particularly within areas of rotation
#ztime = time of the event
#azimuth = direction in which the tornado is moving
merged_df.rename(columns={
    'max_shear': 'wind_speed',
    'wsr_id': 'radar_id',
    'mxdv': 'velocity',
    'ztime': 'event_time',
    'azimuth': 'directional_movement',
}, inplace=True)

# Convert 'ztime' column to datetime
merged_df['event_time'] = pd.to_datetime(merged_df['event_time'])

# Check for missing values
print(merged_df.isnull().sum())

# Remove duplicates
merged_df.drop_duplicates(inplace=True)

# Display the cleaned dataframe
merged_df.head()

In [ ]:
housing_assistance_df

#Made columns names lowercase
housing_assistance_df.columns = housing_assistance_df.columns.str.lower().str.replace(' ', '_')

#dropped unused columns
housing_assistance_df.drop(columns=['id', 
                                    'nofemainspecteddamage', 
                                    'disasternumber',
                                    'validregistrations', 
                                    'averagefemainspecteddamage', 
                                    'totalinspected',
                                   'femainspecteddamagebetween1and10000',
                                    'femainspecteddamagebetween10001and20000',
                                    'femainspecteddamagebetween20001and30000',
                                    'femainspecteddamagegreaterthan30000',
                                    'approvedbetween1and10000',
                                    'approvedbetween10001and25000', 
                                    'approvedbetween25001andmax'], inplace=True)

#renaming columns
#approvedforfemaassistance = number of applications that have been approved for FEMA assistance
#totalapprovedihpamount = total amount of money approved under the FEMA Individual and Households Program (IHP)
#repairreplaceamount = amount of money approved or provided for repair or replacement of damaged property
#rentalamount = amount of money approved or provided for rental assistance
#otherneedsamount = amount of money approved or provided for other necessary expenses
#totalmaxgrants = amount of grants approved or provided to an individual or household

housing_assistance_df.rename(columns={
    'approvedforfemaassistance': 'approvedapplicants',
    'totalapprovedihpamount': 'approvedbudget',
    'repairreplaceamount': 'repairs',
    'rentalamount': 'rentalbudget',
    'otherneedsamount': 'misc.budget',
    'totalmaxgrants': 'grants'
}, inplace=True)

#Gets rid of description in county column
housing_assistance_df['county'] = housing_assistance_df['county'].str.replace(' \(County\)', '', regex=True)

#Sets state and county as index
housing_assistance_df.set_index(['state', 'county'], inplace=True)

#Gets rid of rows in the totaldamage column that have a value of 0
housing_assistance_df[housing_assistance_df['totaldamage'] != 0]

# Remove any NaN values
housing_assistance_df = housing_assistance_df.copy()
housing_assistance_df.dropna(inplace=True)
housing_assistance_df

# Visualization Coding

In [ ]:
import folium
from folium.plugins import HeatMap
from IPython.display import display

try:
    # Check if the DataFrame is not empty
    if merged_df.empty:
        print("The DataFrame is empty. No data to plot.")
    else:
        # Create a base map centered around the mean latitude and longitude
        base_map = folium.Map(location=[merged_df['latitude'].mean(), merged_df['longitude'].mean()], zoom_start=6)

        # Prepare data for the heatmap
        heat_data = [[row['latitude'], row['longitude']] for index, row in merged_df.iterrows()]

        # Add the heatmap to the base map
        HeatMap(heat_data).add_to(base_map)

        # Display the map inline
        display(base_map)
except Exception as e:
    print(f"An error occurred: {e}")


Heat Map: Tornado Activity
Explanation: Start with the heat map to provide a spatial overview of tornado activity. The heat map shows where tornadoes are most frequent across the U.S., using a color gradient to indicate higher concentrations of tornadoes.

Why Important: This visualization sets the stage by identifying regions most affected by tornadoes. It helps understand where tornado activity is most intense, which is crucial for correlating with housing damage and financial assistance.

* Summary: The heat map visualizes tornado activity based on latitude and longitude coordinates. It displays regions with higher concentrations of tornado events using a color gradient.

* Analysis:
The heat map provides a spatial representation of tornado activity across the United States. Areas with higher concentrations of tornadoes are highlighted in warmer colors, indicating regions with more frequent or severe tornado events. This visualization helps in identifying geographic patterns of tornado activity, allowing for targeted preparedness and response efforts. It visually underscores the impact of tornadoes in specific regions, which can be crucial for emergency planning and resource allocation.

* This code generates an interactive heatmap using Folium to visualize the geographical distribution of tornado damage. Each point on the map represents tornado damage locations, with areas of higher damage intensity shown in warmer colors.

* Helps in identifying high-impact regions and understanding spatial patterns of tornado damage across the US. This visual representation is crucial for assessing which areas require more attention or resources.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the DisasterDeclarationsSummaries CSV
disaster_df = pd.read_csv("DisasterDeclarationsSummaries.csv")

# Filter for tornadoes in the US
tornado_df = disaster_df[disaster_df['incidentType'] == 'Tornado']

# Aggregate data by state
state_damage = tornado_df.groupby('state').size().sort_values(ascending=False)

# Plot the bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=state_damage.index, y=state_damage.values, color='steelblue')  # Removed palette, added color
plt.title('Number of Tornadoes by State')
plt.xlabel('State')
plt.ylabel('Number of Tornadoes')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()



Bar Chart: Number of Tornadoes by State
Explanation: Follow with the bar chart displaying the number of tornadoes that occurred in each state. This chart provides a detailed count of tornado occurrences, allowing for a comparison between states.

Why Important: This bar chart adds granularity to the heat map by quantifying tornado events in each state. Comparing this with the heat map helps in understanding the specific tornado activity in high-frequency areas and can be used to correlate with damage and response data.

* This bar chart shows the number of tornadoes that occurred in each state. The states are sorted by the number of tornadoes, providing a clear comparison of tornado frequency across the US.

* Helps to identify which states are most affected by tornadoes. This information is crucial for assessing tornado impact and prioritizing emergency preparedness and response efforts.



In [ ]:
# Convert declarationDate to datetime
disaster_df['declarationDate'] = pd.to_datetime(disaster_df['declarationDate'])

# Group by year
yearly_declarations = disaster_df.groupby(disaster_df['declarationDate'].dt.year).size()

# Plot
plt.figure(figsize=(12, 6))
yearly_declarations.plot(kind='line', marker='o', color='b')
plt.xlabel('Year')
plt.ylabel('Number of Declarations')
plt.title('Disaster Declarations Over Time')
plt.grid(True)
plt.tight_layout()
plt.show()



Line Plot: Trend of Disaster Declarations Over Time
Explanation: Present the line plot showing the trend of disaster declarations over the years. This graph illustrates how the frequency of disaster declarations has changed over time.

Why Important: The line plot provides context on disaster trends and helps link historical data with current tornado activity. It offers insight into whether increased tornado activity corresponds with more disaster declarations, reflecting on data collection and awareness over time.

* This line plot shows the trend of disaster declarations over the years. By grouping the data by year, it visualizes how the frequency of disaster declarations has changed over time.

* Benefit: Useful for identifying trends or patterns in disaster declarations, such as whether certain years had more frequent or severe events. This information can be valuable for analyzing changes in disaster frequency and planning future responses.

In [ ]:
# Aggregate distribution of assistance amounts
assistance_distribution = housing_df[['totalApprovedIhpAmount', 'repairReplaceAmount', 'rentalAmount', 'otherNeedsAmount']].sum()

# Plot
plt.figure(figsize=(10, 7))
assistance_distribution.plot(kind='pie', autopct='%1.1f%%', colors=sns.color_palette('viridis', len(assistance_distribution)))
plt.ylabel('')
plt.title('Distribution of Housing Assistance Amounts')
plt.tight_layout()
plt.show()


Pie Chart: Distribution of Housing Assistance
Explanation: Introduce the pie chart illustrating the distribution of various types of housing assistance. It breaks down how financial resources are allocated among different types of aid.

Why Important: This chart reveals how funds are prioritized for housing assistance, highlighting the focus on comprehensive support and repairs. Understanding this helps in evaluating the effectiveness and allocation of resources in disaster recovery.

* This pie chart illustrates the distribution of various types of housing assistance amounts provided. It shows the proportion of total approved assistance, repair and replace amounts, rental amounts, and other needs amounts.

* **Total Approved IHP Amount**: This represents the total amount of money approved for the **Individuals and Households Program (IHP)**. This program helps people affected by disasters to cover necessary expenses and serious needs that insurance or other forms of assistance don’t cover.

* **Repair/Replace Amount**: This portion of the funds is specifically allocated to repairing or replacing damaged homes.

* **Rental Amount**: This represents funds provided to cover temporary housing expenses while homes are being repaired or replaced.

* **Other Needs Amount**: This includes funds for essential items like medical expenses, transportation, and other serious disaster-related needs.

* Benefit: Provides a clear picture of how housing assistance is allocated among different categories. This helps in understanding which types of assistance are most prevalent and can guide resource allocation and policy-making.

* Analysis:
The pie chart shows that **50% of the housing assistance funds are allocated to the Total Approved IHP Amount**, with 36% directed towards Repair/Replace Amounts.**This indicates a significant focus on broad, comprehensive support through the IHP, ensuring that a wide range of needs are covered for those affected by disasters.** The substantial portion directed towards repairs and replacements highlights the importance of restoring homes, reflecting the critical need for housing stability in disaster recovery. Understanding this distribution helps clarify how financial resources are allocated to support individuals in rebuilding their lives after a disaster.

In [ ]:
# Load the HousingAssistanceOwners CSV
housing_df = pd.read_csv("HousingAssistanceOwners.csv")

# Aggregate damage data by state
state_damage = housing_df.groupby('state')['totalDamage'].sum().sort_values(ascending=False)

# Plot the bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=state_damage.index, y=state_damage.values, color='steelblue')  # Removed palette, added color
plt.title('Total Housing Damage by State')
plt.xlabel('State')
plt.ylabel('Total Damage ($)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


Bar Chart: Total Housing Damage by State
Explanation: Conclude with the bar chart displaying the total housing damage reported by state. This chart shows the financial impact of tornadoes on housing in different states.

Why Important: This bar chart ties together the tornado activity with financial impact, showing which states experience the highest damage. It is crucial for understanding the correlation between tornado frequency and housing damage and assessing the effectiveness of financial assistance.

* This bar chart displays the **total housing damage reported by state.** It aggregates the damage data and visualizes it to show which states experienced the most severe housing damage.

* Provides insights into the distribution of housing damage across states, highlighting areas with the highest financial impacts. This can inform recovery efforts and help allocate resources more effectively.

*  This chart displays the total monetary damage reported in each state due to disasters, particularly focusing on housing.
*  The data is aggregated by state, showing which states experienced the most significant financial impacts from disasters.

**The bar chart illustrates the total housing damage caused by tornadoes in different states. The dollar amounts represent thousands of dollars in damage.**

*  **Ohio (OH) with $7,000: Ohio has the highest total housing damage,** indicating that it experienced the most significant financial impact from tornadoes among the states shown. This substantial amount highlights the severe impact of tornadoes on housing in the state, requiring focused disaster relief and recovery efforts. 

*  **Oklahoma (OK) with $4,000: Oklahoma follows with the second-highest total housing damage.** This amount also represents a significant financial burden due to tornadoes, reflecting the state's vulnerability to severe weather events.\

* Understanding these damage figures is crucial for directing resources and planning effective disaster response strategies. By identifying states with the highest damage, we can better allocate support to areas most in need and enhance preparedness for future tornado events.

# Connecting the Visualizations
**Comparisons and Contrasts:**

**Heat Map vs. Bar Chart (Number of Tornadoes)**: The heat map gives a broad view of tornado activity, while the bar chart provides detailed counts. Together, they offer a comprehensive view of tornado distribution and intensity.

**Bar Chart (Number of Tornadoes) vs. Line Plot**: The bar chart shows the current state of tornado activity, while the line plot shows historical trends. Comparing them can reveal if tornado activity has increased and if this is reflected in disaster declarations.

**Pie Chart vs. Bar Chart (Total Housing Damage)**: The pie chart shows how financial resources are allocated, while the bar chart displays the resulting damage. Comparing these helps evaluate if the distribution of aid is aligned with the regions experiencing the most damage.


**Overall Impact:**

**Understanding Tornado Impact**: The sequence of visualizations helps build a story from where tornadoes are most frequent to how they affect housing and financial responses.


**Effectiveness of Responses**: By comparing tornado activity with housing damage and financial aid distribution, you can assess if the aid is effectively targeted to the most affected areas.


**Why It Matters**:

Disaster Management: This integrated approach supports better disaster management by identifying high-need areas, understanding the effectiveness of financial assistance, and guiding targeted interventions for future tornado events.

# Analysis of Disaster Declarations and Housing Assistance

## 1. What trends are evident in the number of disaster declarations over time?

**Answer**:  
The line chart reveals that there were peaks in the number of disaster declarations in the years 1985, 1995, 2002, and 2021. Conversely, the years 1980, 1990, and 2005 saw the lowest number of declarations.

**Analysis**:  
The peaks in disaster declarations could indicate periods of increased disaster activity or better data reporting and awareness. The years with lower numbers might reflect fewer significant disasters or less comprehensive data collection during those times. This trend analysis helps identify periods of increased disaster activity and evaluate how reporting practices or disaster frequency have changed over time.

## 2. How are housing assistance funds distributed across different types of aid?

**Answer**:  
The pie chart shows that 50% of housing assistance funds are allocated to the Total Approved IHP Amount, while 36% are directed towards Repair/Replace Amounts.

**Analysis**:  
The distribution indicates that a significant portion of funds is used for comprehensive housing assistance (Total Approved IHP Amount), suggesting a focus on broad support measures. The Repair/Replace Amount also represents a substantial portion, highlighting the importance of repairing and replacing damaged properties. This distribution helps understand the priorities in housing assistance and how financial resources are allocated to support affected individuals.

## 3. Which states experience the highest number of tornadoes?

**Answer**:  
The bar chart indicates that the states with the highest number of tornadoes are Georgia (GA), Oklahoma (OK), Illinois (IL), Tennessee (TN), Kentucky (KY), Arkansas (AR), and Mississippi (MS).

**Analysis**:  
These states are located in regions known for high tornado activity, such as the "Tornado Alley" in the central U.S. The data reflects their susceptibility to tornadoes, which can be attributed to their geographic location and climatic conditions. Identifying these states helps in understanding tornado-prone areas and allocating resources for tornado preparedness and response.

## 4. Which states report the highest total housing damage?

**Answer**:  
The bar chart shows that Ohio (OH) and Oklahoma (OK) report the highest total housing damage.

**Analysis**:  
The significant damage reported in these states suggests either frequent or severe disaster events in these regions. Ohio and Oklahoma could have experienced major disasters or a high frequency of impactful events, leading to substantial housing damage. Understanding these areas with high damage levels helps target relief efforts and improve disaster response strategies.

## 5. How does the heat map illustrate tornado activity?

**Answer**:  
The heat map visualizes tornado activity based on latitude and longitude coordinates, highlighting regions with higher tornado frequencies using a color gradient.

**Analysis**:  
The heat map provides a spatial view of tornado activity, with warmer colors indicating areas of higher tornado concentrations. This visualization helps identify tornado-prone regions and is useful for understanding the geographic distribution of tornadoes. It ties together with other data by showing where the most tornadoes occur, helping to focus disaster preparedness and response efforts in these areas.

## Connecting the Insights

The combination of these charts offers a detailed view of disaster impacts and response. The line chart on disaster declarations shows the evolution of disaster activity and reporting over time. The pie chart on housing assistance funds reveals how financial resources are allocated to support affected individuals. The bar charts on tornado frequency and housing damage provide insight into the geographical distribution and financial impact of tornadoes.

The heat map ties these elements together by visually representing the spatial distribution of tornado activity. By comparing this with the financial and frequency data, we can better understand how tornado activity correlates with housing damage and resource allocation. This integrated analysis supports more effective disaster management strategies, highlighting areas of high need and guiding targeted interventions.

## Integrated Analysis

- **Combining Data**: By integrating insights from these visualizations, you can analyze disaster impacts and responses comprehensively.
- **Trends**: The line chart shows how disaster activity has evolved.
- **Financial Allocation**: The pie chart reveals how funds are distributed.
- **Geographical Patterns**: Bar charts and heat maps provide context on tornado activity and housing damage.
- **Correlation**: Comparing tornado activity with housing damage and financial data helps understand the relationship between disaster frequency, impact, and resource allocation.

This integrated approach supports effective disaster management by identifying high-need areas and guiding targeted interventions.
